In [1]:
pip install opencv-python numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 2.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
#importation des librairies
import os
import cv2
import numpy as np
import json

In [5]:
# Définition de la fonction pour convertir un fichier JSON en masque

#----------#
def json_to_mask(json_file):
    with open(json_file, 'r') as f: #ouvre fichier json en lecture "r" read
        data = json.load(f)

    # Initialise le masque avec des valeurs de pixel différentes pour chaque label
    mask = np.zeros((data['imageHeight'], data['imageWidth']), dtype=np.uint8)

    # Parcourt chaque forme dans le fichier JSON
    for shape in data['shapes']:
        label = shape['label']
        points = shape['points']
        polygon = np.array(points, dtype=np.int32)

        # Définir une valeur de pixel différente pour chaque label
        if label == 'numeroex':
            cv2.fillPoly(mask, [polygon], 128)  # Par exemple, utilisez 128 pour "numeroex"
        elif label == 'Enonce':
            cv2.fillPoly(mask, [polygon], 255)  # Par exemple, utilisez 255 pour "Enonce"
        elif label == 'tab':
            cv2.fillPoly(mask, [polygon], 64)   # Par exemple, utilisez 64 pour "tab"
        elif label == 'img':
            cv2.fillPoly(mask, [polygon], 192)  # Par exemple, utilisez 192 pour "img"

    return mask
#----------#

#----------#
# Chemin vers le répertoire contenant les fichiers JSON
quentin_JSON = '/Users/quentinlandon/Desktop/GIT_Projet_theo/APP_IA_TL_QL_dev/data/label'
# Chemin vers le répertoire contenant les images brutes associées
quentin_BRUT = '/Users/quentinlandon/Desktop/GIT_Projet_theo/APP_IA_TL_QL_dev/data/Image'
# Chemin vers le répertoire où vous souhaitez enregistrer les masques
quentin_mask = '/Users/quentinlandon/Desktop/GIT_Projet_theo/APP_IA_TL_QL_dev/data/mask'

theo_JSON = '/Users/theolambs/APP_IA_TL_QL_dev/data/label'
theo_BRUT = '/Users/theolambs/APP_IA_TL_QL_dev/data/Image'
theo_mask = '/Users/theolambs/APP_IA_TL_QL_dev/data/mask'

#Choix de l'utilisateur
utilisateur = int(input("si quentin tape 0, si theo tape 1"))

if utilisateur == 0:
    json_dir = quentin_JSON
    img_dir = quentin_BRUT
    mask_dir = quentin_mask
else :
    json_dir = theo_JSON
    img_dir = theo_BRUT
    mask_dir = theo_mask

#----------#


# Assurez-vous que le répertoire pour les masques existe, sinon créez-le
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)

# Parcours de tous les fichiers JSON dans le répertoire
#----------#
for json_file in os.listdir(json_dir):
    if json_file.startswith('LEnonce') and json_file.endswith('.json'):
        # Obtient le numéro de fichier de l'image correspondante
        image_number = json_file[7:-5]  # Supprime "LEnonce" du début et ".json" de la fin
        
        # Crée le nom de fichier de l'image
        img_filename = f'Enonce{image_number}.png'
        
        # Chemin complet vers le fichier d'image
        img_path = os.path.join(img_dir, img_filename)
        
        # Vérifie si l'image associée existe
        if os.path.exists(img_path):
            # Convertit le fichier JSON en masque
            mask = json_to_mask(os.path.join(json_dir, json_file))
            
            # Enregistre le masque avec le même nom que le fichier JSON
            mask_filename = os.path.splitext(json_file)[0] + '.png'
            mask_file_path = os.path.join(mask_dir, mask_filename)
            cv2.imwrite(mask_file_path, mask)
            
            # Vérifie si le fichier de masque a été correctement enregistré
            if os.path.exists(mask_file_path):
                print(f"Masque enregistré avec succès : {mask_file_path}")
            else:
                print(f"Erreur lors de l'enregistrement du masque : {mask_file_path}")
        else:
            print(f"Image associée non trouvée pour : {json_file}")
#----------#




Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce37.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce136.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce60.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce161.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce99.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce76.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce120.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce7.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce83.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce95.png
Masque enregistré avec succès : /Users/theolambs/APP_IA_TL_QL_dev/data/mask/LEnonce157.png
Masque 